<a href="https://colab.research.google.com/github/Agil0509/NLP/blob/main/MakeMore3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt

In [ ]:
words = open('/content/names.txt','r').read().splitlines()
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [ ]:
len(words)

32033

In [ ]:
chars = sorted(set(''.join(words)))
stoi = {s: i+1 for i,s in enumerate(chars)}

stoi['.'] = 0
itos = {i: s for s,i in stoi.items()}
vocab_size = len(itos)
print(itos)
print(vocab_size)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}
27


In [ ]:
block_size = 3
def build_dataset(words):

  X, Y = [], []
  for  w in words:
    context = [0] * block_size # 3 dene nokte
    for ch in w + '.':
        ix = stoi[ch]
        X.append(context)
        Y.append(ix)
        # print(''.join(itos[i]for i in context), '--->',itos[ix])
        context = context[1:] + [ix]
  X = torch.tensor(X)
  Y = torch.tensor(Y)
  print(X.shape,Y.shape)
  return X,Y

import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr,Ytr = build_dataset(words[:n1])
Xdev,Ydev = build_dataset(words[n1:n2])
Xte,Yte = build_dataset(words[n2:])

torch.Size([182625, 3]) torch.Size([182625])
torch.Size([22655, 3]) torch.Size([22655])
torch.Size([22866, 3]) torch.Size([22866])


In [ ]:
import torch

# Define missing variables
vocab_size = 27  # Example: Alphabet size
block_size = 3   # Example: Input sequence length
n_embd = 10
n_hidden = 200

# Initialize generator for reproducibility
g = torch.Generator().manual_seed(42)

# Parameter initialization
C = torch.rand((vocab_size, n_embd), generator=g)  # Embedding matrix
W1 = torch.randn((n_embd * block_size, n_hidden), generator=g) * (5 / 3) / (n_embd * block_size) ** 0.5
b1 = torch.randn(n_hidden, generator=g) * 0.01
W2 = torch.randn((n_hidden, vocab_size), generator=g) * 0.1
b2 = torch.randn(vocab_size, generator=g)* 0 # Bias initialized to 0

bngain= torch.ones((1,n_hidden))
bnbias= torch.zeros((1,n_hidden))
bnmean_running = torch.zeros((1,n_hidden))
bnstd_running = torch.ones((1,n_hidden))
# List of parameters for optimization
parametrs = [C, W1, b1, W2, b2,bngain,bnbias]

# Print total number of parameters
print(f"Total parameters: {sum(p.nelement() for p in parametrs)}")

# Set requires_grad = True for all parameters
for p in parametrs:
    p.requires_grad = True


Total parameters: 12297


In [ ]:
vocab_size

27

In [ ]:
W2.shape

torch.Size([200, 27])

In [ ]:
max_steps = 200000
batch_size = 32
lossi = []
for i in range(max_steps):

  ix = torch.randint(0, Xtr.shape[0], (batch_size,), generator=g)

  Xb,Yb = Xtr[ix],Ytr[ix]

  emb = C[Xb]
  embcat = emb.view(emb.shape[0],-1)
  hpreact = embcat @ W1 #+b1

  bnmeani = hpreact.mean(0,keepdim = True)
  bnstdi = hpreact.std(0,keepdim = True)
  hpreact =bngain*(hpreact-bnmeani)/bnstdi + bnbias
  with torch.no_grad():
    bnmean_running = 0.999 *bnmean_running + 0.001*bnmeani
    bnstd_running = 0.999 *bnstd_running + 0.001*bnstdi

  h = torch.tanh(hpreact)
  logits = h @ W2 + b2
  loss = F.cross_entropy(logits,Yb)


  for p in parametrs:
    p.grad = None
  loss.backward()

  lr  = 0.1 if i <100000 else 0.01

  for p in parametrs:
    p.data += -lr *p.grad
  if i % 10000 == 0 :
    print(f'{i:7d}/{max_steps:7d}:{loss.item():.4f}')
  lossi.append(loss.log10().item())



TypeError: unsupported operand type(s) for *: 'float' and 'NoneType'

In [ ]:
plt.figure(figsize = (10,20))
plt.imshow(h.abs()>0.99,cmap = 'gray',interpolation='nearest')

In [ ]:
plt.hist(h.view(-1).tolist(),50);

In [ ]:
plt.plot(lossi)

In [ ]:
with torch.no_grad():
  emb  = C[Xtr]
  embcat = emb.view(emb.shape[0],-1)
  hpreact = embcat @ W1 +b1
  bnmean = hpreact.mean(0,keepdim = True)
  bnstd = hpreact.std(0,keepdim = True)

In [ ]:
@torch.no_grad()
def split_loss(split):
    x, y = {
        'train': (Xtr, Ytr),
        'val': (Xdev, Ydev),
        'test': (Xte, Yte)
    }[split]
    emb = C[x]  # (batch_size, block_size, n_embd)
    embcat = emb.view(emb.shape[0], -1)  # (batch_size, n_embd * block_size)
    hpreact = embcat @ W1 + b1  # (batch_size, n_hidden)
    # hpreact =bngain * (hpreact-hpreact.mean(0,keepdim = True))/hpreact.std(0,keepdim = True) + bnbias
    hpreact =bngain * (hpreact-bnmean_running)/bnstd_running + bnbias
    h = torch.tanh(hpreact)      # Apply non-linearity
    logits = h @ W2 + b2  # (batch_size, vocab_size)
    loss = F.cross_entropy(logits, y)
    print(split, loss.item())

split_loss('train')
split_loss('val')



In [ ]:
g = torch.Generator().manual_seed(2147483647 + 10)

for _ in range(20):

    out = []
    context = [0] * block_size
    while True:
      # forward pass the neural net
      emb = C[torch.tensor([context])]
      embcat = emb.view(emb.shape[0], -1)
      hpreact = embcat @ W1 + b1
      hpreact =bngain * (hpreact-bnmean_running)/bnstd_running + bnbias
      h = torch.tanh(hpreact)      # Apply non-linearity
      logits = h @ W2 + b2
      probs = F.softmax(logits, dim=1)
      ix = torch.multinomial(probs, num_samples=1, generator=g).item()
      context = context[1:] + [ix]
      out.append(ix)
      if ix == 0:
        break

    print(''.join(itos[i] for i in out))

In [ ]:
class Linear:
  def __init__(self,fan_in,fan_out,bias = True):
    self.weight = torch.randn((fan_in,fan_out), generator = g)/ fan_in**0.5
    self.bias = torch.zeros(fan_out) if bias else None


  def __call__(self,x):
    self.out = x@ self.weight
    if self.bias is not None:
      self.out += self.bias
    return self.out

  def parametrs(self):
    return [self.weight] + ([] if self.bias is None else [self.bias])


class BatchNorm1d:
  def __init__(self,dim,eps  =1e-5,momentum = 0.01):
    self.eps = eps
    self.momentum = momentum
    self.training = True
    self.gamma = torch.ones(dim)
    self.beta = torch.zeros(dim)
    self.running_mean = torch.zeros(dim)
    self.running_var = torch.ones(dim)

  def __call__(self,x):
    if self.tarining:
      xmean = x.mean(0,keepdim = True)
      xvar = x.var(0,keepdim = True)
    else:
        xmean = self.running_mean
        xvar = self.ruuning_var

    xhat = (x-xmean)/torch.sqrt(xvar+self.eps)
    self.out = self.gamma * xhat +self.beta

    if self.training:
      with torch.no_grad():
        self.running_mean = (1-self.momentum) * self.running_mean + self. momentum * xmean
        self.running_var = (1-self.momentum) * self.running_var + self. momentum * xvar
    return self.fan_out

  def parametrs(self):
    return [self.gamma,self.beta]


class Tanh:
  def __call__(self,x):
    self.out = torch.tanh(x)
    return self.out

  def parametrs(self):
    return []


n_embd  = 10
n_hidden  = 100
g = torch.Generator().manual_seed(42)

C = torch.randn((vocab_size,n_embd),   generator = g)


layers = [
    Linear(n_embd*block_size,n_hidden),Tanh(),
    Linear(n_hidden,n_hidden),Tanh(),
    Linear(n_hidden,n_hidden),Tanh(),
    Linear(n_hidden,n_hidden),Tanh(),
    Linear(n_hidden,n_hidden),Tanh(),
    Linear(n_hidden,vocab_size),
]

with  torch.no_grad():
  layers[-1].weight *= 0.1
  for layer in layers[:-1]:
    if isinstance(layer,Linear):
      layer.weight *= 5/3


parametrs = [C] + [p for layer in layers for p in layer.parametrs()]
print(sum(p.nelement() for p in parametrs))

for p in parametrs:
  p.requires_grad = True